In [1]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from tqdm import tqdm

from pangres import upsert
from sqlalchemy import text, create_engine

load_dotenv()

True

In [2]:
db_username=os.environ.get("db_username")
db_password=os.environ.get("db_password")
db_host=os.environ.get("db_host")
db_port=os.environ.get("db_port")
# db_name=os.environ.get("db_name")
db_name = 'lol_analytics'


def create_db_connection_string(db_username: str, db_password: str, db_host: str, db_port: int, db_name: str):
    connection_url = f"postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}"
    return connection_url

In [3]:
def get_puuid(username, tagline, api_key):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{username}/{tagline}?api_key={api_key}'

    response = requests.get(link)
    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print(f"Error: {response.status_code}")
    return None

In [17]:
uhe_varsity = [
    {
        'username': 'hawnlilmonster',
        'tagline': '0528',
        'team_position': 'TOP',
        'is_varsity': True
    },
    {
        'username': 'bruhdun',
        'tagline': '249',
        'team_position': 'JUNGLE',
        'is_varsity': True,
    },
    {
        'username': 'oanduh',
        'tagline': 'oan',
        'team_position': 'MIDDLE',
        'is_varsity': True,
    },
    {
        'username': 'RoboBoto',
        'tagline': 'NA1',
        'team_position': 'BOTTOM',
        'is_varsity': True,
    },
    {
        'username': 'mazsu',
        'tagline': 'mas',
        'team_position': 'UTILITY',
        'is_varsity': True,
    }, 
]



In [18]:
api_key = os.getenv("riot_api_key")
def process_player_data(uhe_varsity, api_key):

    for player in uhe_varsity:
        username = player['username']
        tagline = player['tagline']
        puuid = get_puuid(username, tagline, api_key)
        player['puuid'] = puuid

    return uhe_varsity

uhe_varsity = process_player_data(uhe_varsity, api_key)

In [19]:
user_metadata_df = pd.DataFrame(uhe_varsity).set_index('puuid')
user_metadata_df

,username,tagline,team_position,is_varsity
puuid,,,,
ZzKy705E6fkzFEscD6HBqypYH3EWriEgumWJPgH-RU2l-NK-Zg40flY03DB-JxmuUWtlTGkzQYRqHw,hawnlilmonster,0528,TOP,True
_rpt4no7mbwiPc8L1-gacv4tvLjI0giGdBtKlc2ZCw8pN7BQjvzjbjw2ZlfZijiZMzAfRODlrN_KCQ,bruhdun,249,JUNGLE,True
jgIFggFSUWMaiXjdREgaDpoQOcs5uyzwaS8tm781QGJtJHJazhaY1xPG5z9uUtSEy8CKDzQodTmTGQ,oanduh,oan,MIDDLE,True
E-19GsPkwE-eUzeDs-R-CXe6QhA8Sykijb5jOib8It2_J1viqlVprlWOc4rT_2g6JnhHFeRY5tELrQ,RoboBoto,NA1,BOTTOM,True
1ZcFop5pM6PFXkveK0sJHoSIoJh53fHm-kF1MR3_zS1Rp7V71wA-u8y_8X6ivwb7P4jUuXUE8c6H7Q,mazsu,mas,UTILITY,True


In [20]:
connection_url = create_db_connection_string(db_username, db_password, db_host, db_port, db_name)
db_engine = create_engine(connection_url, pool_recycle=3600)
connection = db_engine.connect()
upsert(con=connection, df=user_metadata_df, schema='soloq', table_name='players', create_schema=True, if_row_exists='update')

In [21]:
connection.commit()